In [29]:
from Bio import Align
from Bio.Align import substitution_matrices
#lookup https://biopython.org/docs/dev/Tutorial/chapter_pairwise.html

# GLOBAL ALLIGMENT USING BIOPYTHON

In [30]:
# GLOBAL ALIGNMENT
aligner = Align.PairwiseAligner()
aligner.mode = "global"
print("Alignment mode:", aligner.algorithm)
aligner.mismatch_score = -1
print("Global Alignment:")
alignments = aligner.align("AAACAAA", "AAAGAAA")
for alignment in alignments:
    print(alignment)
#matrix = substitution_matrices.load("BLOSUM62")
#aligner.substitution_matrix = matrix
#print("Global Alignment with BLOSUM62:")
#print(matrix)


Alignment mode: Needleman-Wunsch
Global Alignment:
target            0 AAAC-AAA 7
                  0 |||--||| 8
query             0 AAA-GAAA 7

target            0 AAA-CAAA 7
                  0 |||--||| 8
query             0 AAAG-AAA 7



# LOCAL ALLIGNMENT USING BIOPYTHON

In [31]:
#local alignment
aligner.mode = "local"
print("Alignment mode:", aligner.algorithm)
print("Local Alignment:")
alignments = aligner.align("GCCGATAAGCAC", "GCCGAGAAGCAC")
for alignment in alignments:
    print(alignment)


Alignment mode: Smith-Waterman
Local Alignment:
target            0 GCCGAT-AAGCAC 12
                  0 |||||--|||||| 13
query             0 GCCGA-GAAGCAC 12

target            0 GCCGA-TAAGCAC 12
                  0 |||||--|||||| 13
query             0 GCCGAG-AAGCAC 12



# Smith waterman without using Biopython (LOCAL ALLIGMENT)

In [32]:
def smith_waterman(seq1, seq2, match=2, mismatch=-1, gap_open=0, gap_extend=-1):
    # Initialize scoring matrix
    n, m = len(seq1), len(seq2)
    H = [[0] * (m + 1) for _ in range(n + 1)]
    # Traceback matrix
    traceback = [[0] * (m + 1) for _ in range(n + 1)]
    max_score = 0
    max_pos = None
    # Fill matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            # Match/mismatch
            score_diag = H[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            # Gap in seq1
            score_up = H[i-1][j] + (gap_extend if traceback[i-1][j] == 1 else gap_open)
            # Gap in seq2
            score_left = H[i][j-1] + (gap_extend if traceback[i][j-1] == 2 else gap_open)          
            # Local alignment: no negative scores
            H[i][j] = max(0, score_diag, score_up, score_left)       
            # Track direction
            if H[i][j] == score_diag:
                traceback[i][j] = 3  # diagonal
            elif H[i][j] == score_up:
                traceback[i][j] = 1  # up
            elif H[i][j] == score_left:
                traceback[i][j] = 2  # left      
            # Track max score
            if H[i][j] >= max_score:
                max_score = H[i][j]
                max_pos = (i, j)
    # Print scoring matrix
    print("Scoring Matrix (H):")
    header = "      " + "  ".join(seq2)
    print(header)
    for i in range(n + 1):
        row = (seq1[i-1] if i > 0 else " ") + " " + " ".join(f"{H[i][j]:2}" for j in range(m + 1))
        print(row)
    # Traceback to get alignment
    aligned1, aligned2 = "", ""
    i, j = max_pos
    while H[i][j] > 0:
        if traceback[i][j] == 3:
            aligned1 = seq1[i-1] + aligned1
            aligned2 = seq2[j-1] + aligned2
            i -= 1
            j -= 1
        elif traceback[i][j] == 1:
            aligned1 = seq1[i-1] + aligned1
            aligned2 = "-" + aligned2
            i -= 1
        elif traceback[i][j] == 2:
            aligned1 = "-" + aligned1
            aligned2 = seq2[j-1] + aligned2
            j -= 1

    return max_score, aligned1, aligned2
# Example usage
seq1 = "GCCGATAAGCAC"
seq2 = "GCCGAGAAGCAC"

score, aln1, aln2 = smith_waterman(seq1, seq2)
print("Alignment score:", score)
print(aln1)
print(aln2)


Scoring Matrix (H):
      G  C  C  G  A  G  A  A  G  C  A  C
   0  0  0  0  0  0  0  0  0  0  0  0  0
G  0  2  2  1  2  2  2  2  1  2  2  1  0
C  0  2  4  4  4  3  2  2  2  2  4  4  3
C  0  1  4  6  6  5  4  3  2  2  4  4  6
G  0  2  4  6  8  8  7  7  6  5  4  4  6
A  0  2  3  5  8 10 10  9  9  9  8  7  6
T  0  1  2  4  7 10 10 10  9  9  9  8  7
A  0  0  1  3  6  9  9 12 12 12 11 11 11
A  0  0  0  2  5  9  9 12 14 14 13 13 13
G  0  2  2  1  4  8 11 11 14 16 16 15 14
C  0  2  4  4  4  7 11 11 13 16 18 18 17
A  0  1  4  4  4  6 10 13 13 15 18 20 20
C  0  0  3  6  6  6  9 13 13 14 17 20 22
Alignment score: 22
GCCGA-TAAGCAC
GCCGAG-AAGCAC


# Needleman wunch algorith without using Biopython(GLOBAL ALLIGMENT)

In [33]:
def needleman_wunsch(seq1, seq2, match=2, mismatch=-1, gap_open=0, gap_extend=-2):
    n, m = len(seq1), len(seq2)
    H = [[0] * (m + 1) for _ in range(n + 1)]
    traceback = [[0] * (m + 1) for _ in range(n + 1)]

    # Initialize first row/column with gap penalties
    for i in range(1, n + 1):
        H[i][0] = gap_open + (i - 1) * gap_extend
        traceback[i][0] = 1
    for j in range(1, m + 1):
        H[0][j] = gap_open + (j - 1) * gap_extend
        traceback[0][j] = 2

    # Fill matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            score_diag = H[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            score_up = H[i-1][j] + (gap_extend if traceback[i-1][j] == 1 else gap_open)
            score_left = H[i][j-1] + (gap_extend if traceback[i][j-1] == 2 else gap_open)

            H[i][j] = max(score_diag, score_up, score_left)

            if H[i][j] == score_diag:
                traceback[i][j] = 3
            elif H[i][j] == score_up:
                traceback[i][j] = 1
            elif H[i][j] == score_left:
                traceback[i][j] = 2

    # Print scoring matrix
    print("Scoring Matrix (H):")
    header = "       " + "   ".join(seq2)
    print(header)
    for i in range(n + 1):
        row = (seq1[i-1] if i > 0 else " ") + " " + "  ".join(f"{H[i][j]:2}" for j in range(m + 1))
        print(row)

    # Traceback from bottom-right
    aligned1, aligned2 = "", ""
    i, j = n, m
    while i > 0 or j > 0:
        if traceback[i][j] == 3:
            aligned1 = seq1[i-1] + aligned1
            aligned2 = seq2[j-1] + aligned2
            i -= 1; j -= 1
        elif traceback[i][j] == 1:
            aligned1 = seq1[i-1] + aligned1
            aligned2 = "-" + aligned2
            i -= 1
        elif traceback[i][j] == 2:
            aligned1 = "-" + aligned1
            aligned2 = seq2[j-1] + aligned2
            j -= 1
    return H[n][m], aligned1, aligned2


# Example usage
seq1 = "ACACACTA"
seq2 = "AGCACACA"

score, aln1, aln2 = needleman_wunsch(seq1, seq2)
print("\nGlobal Alignment Score:", score)
print(aln1)
print(aln2)


Scoring Matrix (H):
       A   G   C   A   C   A   C   A
   0   0  -2  -4  -6  -8  -10  -12  -14
A  0   2   2   0  -2  -2  -4  -6  -8
C -2   2   2   4   4   2   0  -2  -2
A -4   0   1   4   6   6   4   4   2
C -6   0   1   3   6   8   8   6   6
A -8  -2  -1   3   5   8  10  10   8
C -10  -4  -1   1   5   7  10  12  12
T -12  -6  -3   1   3   7   8  12  12
A -14  -8  -5  -1   3   5   9  10  14

Global Alignment Score: 14
A-CACACTA
AGCACAC-A
